In [1]:
%matplotlib qt
#import mdp
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#load data set (168 coercivity distributions from Lunca interpolated to a common field)

data = np.genfromtxt("Lunca_coercivity_dist.txt")
data.shape

(29, 168)

In [3]:
#load the interpolation field

h = np.genfromtxt("field_TM_irm.txt")
h=h.T
h

array([9.994200e-01, 2.201770e+00, 3.643830e+00, 5.374030e+00,
       7.455770e+00, 9.952980e+00, 1.295411e+01, 1.654645e+01,
       2.087193e+01, 2.607686e+01, 3.228113e+01, 3.975521e+01,
       4.875319e+01, 5.955447e+01, 7.255609e+01, 8.814643e+01,
       1.068138e+02, 1.292536e+02, 1.562369e+02, 1.886414e+02,
       2.274950e+02, 2.742648e+02, 3.301602e+02, 3.975829e+02,
       4.782787e+02, 5.755419e+02, 6.918658e+02, 8.321459e+02,
       1.000170e+03])

In [4]:
# plot the coercivity distributions

for i in range(len(data.T)):
    plt.plot(np.log10(h),data[:,i])

In [5]:
# Transpose the data and Normalize the coercivity distribution to sum to 1

data1=data.T
data1.shape

dataPn=np.zeros((168,29))
for i in range(len(data1)):
    dataPn[i,:]=(data1[i,:])/(np.sum(data1[i,:]))
print(dataPn.shape)
for i in range(len(data1)):
    plt.plot(np.log10(h),dataPn[i,:])

(168, 29)


In [6]:
# Load the coercivity distributions into the hyperspy

s=hs.signals.Signal1D(dataPn)
#s=hs.signals.Signal1D(data1)
s

<Signal1D, title: , dimensions: (168|29)>

In [7]:
# SVD decomposition and plot the explained variance ratio

s.decomposition()#algorithm='MLPCA',output_dimension=2)#centre='signal')
#s.plot_explained_variance_ratio(n=10)

Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SVD
  output_dimension=None
  centre=None


In [8]:
# determine the number of end-members by calculating the cumulative variance given by SVD

a=s.get_explained_variance_ratio()
var=a.data
#var
varnorm=np.cumsum(var)
varnorm
pc_idx=np.linspace(1,10,10)
plt.plot(pc_idx,varnorm[0:10]*100,'o')
plt.xticks(np.arange(1,11,step=1))
plt.xlabel('Principal Component index')
plt.ylabel('Cumulative Variance (%)')

Text(0, 0.5, 'Cumulative Variance (%)')

In [9]:
# import Fastica package

from sklearn.decomposition import FastICA

In [23]:
# apply fastica algorithm to the first 2 principal components resulted from SVD

s.blind_source_separation(number_of_components=2,algorithm=FastICA(algorithm='parallel',random_state=1),diff_order=3,fun='exp')

[########################################] | 100% Completed |  0.1s
Blind source separation info:
  number_of_components=2
  algorithm=FastICA(random_state=1)
  diff_order=3
  reverse_component_criterion=factors
  whiten_method=PCA
scikit-learn estimator:
FastICA(random_state=1)


In [25]:
s.plot_bss_results()

In [26]:
# plot EM1

factors=s.get_bss_factors()
em1=factors.inav[0].data
plt.plot(h,em1)
plt.xscale('log')

In [27]:
# plot EM2

em2=factors.inav[1].data
plt.plot(h,em2)
plt.xscale('log')

In [1]:
# calculate and plot the relative contributions. This reproduce Fig. 6b

loadings=s.get_bss_loadings()
em1_l=loadings.inav[0].data
em2_l=loadings.inav[1].data
#em1_l=em1_l+np.abs(np.min(em2_l))
#em2_l=em2_l-np.min(em2_l)
#em2_l=np.where(em2_l < 0, -1*em2_l, em2_l)
#print(em2_l)
depth=np.genfromtxt('Lunca_depth_IRM.txt')
#depth=cirm[:,0]
em2ln=em2_l/(em1_l+em2_l)
em1ln=em1_l/(em1_l+em2_l)
plt.stackplot(depth,em2ln[:-1],em1ln[:-1],labels=('EM1','EM2'))
plt.axhline(y=0.5,color='white')

NameError: name 's' is not defined

In [29]:
# put both EMs on the same graph. This reproduce Fig. 6a

#em11=np.cumsum(em1*np.diff(np.log10(h1)))*-1
#em22=np.cumsum(em2*np.diff(np.log10(h1)))*-1
#print(np.min(em22/np.max(em22)))
#print(np.min(em11))
plt.plot(h,em2/np.sum(em2),label='EM1')
plt.plot(h,em1/np.sum(em1),label='EM2')
plt.text(6,0.1,'~21mT',fontsize=15)
plt.text(40,0.09,'~60mT',fontsize=15)
plt.legend(loc='upper right')
plt.xscale('log')

C:\Users\cristian\anaconda3\envs\Hyperspy_Preisach\lib\site-packages\matplotlib\colors.py:263: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  c = np.array(c)
